In [1]:
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D

2024-09-29 19:30:38.572306: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-29 19:30:38.579468: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-29 19:30:38.601914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 19:30:38.638802: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 19:30:38.649462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 19:30:38.675162: I tensorflow/core/platform/cpu_feature_gu

In [2]:
import tensorflow as tf

In [3]:
# Function to load and preprocess images
def load_images_from_directory(path, image_size=(32, 32)):
    images = os.listdir(path)
    data = []
    labels = []
    class_id = int(path.split('/')[-1])

    for image_filename in images:
        try:
            image = Image.open(os.path.join(path, image_filename))
            image = image.resize(image_size)
            image = np.array(image)
            data.append(image)
            labels.append(class_id)
        except:
            print(f"Error loading image: {os.path.join(path, image_filename)}")

    return data, labels

# Retrieving the images and their labels
data = []
labels = []
num_classes = 43
current_path = os.getcwd()

for class_id in range(num_classes):
    path = os.path.join(current_path, '/media/sathish/#sathish1/Trafic sign classification data/Data', str(class_id))
    class_data, class_labels = load_images_from_directory(path)
    data.extend(class_data)
    labels.extend(class_labels)

# Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)

(39209, 32, 32, 3) (39209,)


In [5]:
# Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31367, 32, 32, 3) (7842, 32, 32, 3) (31367,) (7842,)


In [7]:
# Converting the labels into one hot encoding
y_train_one_hot = to_categorical(y_train, num_classes)
y_test_one_hot = to_categorical(y_test, num_classes)
print(y_train_one_hot)
print(y_test_one_hot)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
inp = Input(shape=(32, 32, 3))
# Define the layers
x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(inp)
x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=2, strides=2, padding='valid')(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=2, strides=2, padding='valid')(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
x = Dense(units=64, activation='relu')(x)
x = Dense(units=43, activation='softmax')(x)

# Create the model
model_custom_cnn = Model(inputs=inp, outputs=x)

# Print the model summary
model_custom_cnn.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       262,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 43)             │         2,795 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 330,571 (1.26 MB)

 Trainable params: 330,571 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:

# Compilation of the model
model_custom_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 15
history = model_custom_cnn.fit(X_train, y_train_one_hot, batch_size=32, epochs=epochs, validation_data=(X_test, y_test_one_hot))

Epoch 1/15


2024-09-29 20:01:42.466775: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 96359424 exceeds 10% of free system memory.
2024-09-29 20:01:49.223577: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23592960 exceeds 10% of free system memory.
2024-09-29 20:01:49.302947: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 21528576 exceeds 10% of free system memory.


981/981 ━━━━━━━━━━━━━━━━━━━━ 295s 295ms/step - accuracy: 0.2325 - loss: 3.5039 - val_accuracy: 0.9245 - val_loss: 0.2906
Epoch 2/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 327s 301ms/step - accuracy: 0.9130 - loss: 0.3116 - val_accuracy: 0.9718 - val_loss: 0.1167
Epoch 3/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 328s 307ms/step - accuracy: 0.9566 - loss: 0.1573 - val_accuracy: 0.9806 - val_loss: 0.0779
Epoch 4/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 304s 310ms/step - accuracy: 0.9695 - loss: 0.1158 - val_accuracy: 0.9842 - val_loss: 0.0780
Epoch 5/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 312s 300ms/step - accuracy: 0.9743 - loss: 0.0967 - val_accuracy: 0.9839 - val_loss: 0.0784
Epoch 6/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 326s 305ms/step - accuracy: 0.9782 - loss: 0.0757 - val_accuracy: 0.9674 - val_loss: 0.1451
Epoch 7/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 322s 305ms/step - accuracy: 0.9695 - loss: 0.1160 - val_accuracy: 0.9892 - val_loss: 0.0494
Epoch 8/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 300s 306ms/step - accuracy: 0.9845 - loss: 0.06

In [20]:
model_custom_cnn.save('traffic_classifier_model5.h5')